In [167]:
import tensorflow as tf
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [175]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.01, shape=shape))

In [176]:
def conv2d(x, W, stride_size=2):
    return tf.nn.conv2d(x, W, strides=[1,stride_size,stride_size,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [213]:
WIDTH = 640 # pixel width of each image
HEIGHT = 640 # pixel height of each image

x = tf.placeholder(tf.float32, shape=[None, WIDTH*HEIGHT]) # input is flattened image
y = tf.placeholder(tf.float32, shape=[None, 1]) # output is one value

x_image = tf.reshape(x, [-1, WIDTH, HEIGHT, 1])

# Weights and biases between input and hidden layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Hidden layer 1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, stride_size=4) + b_conv1)
# Max pooled hidden layer 1
h_pool1 = max_pool_2x2(h_conv1) #dim is 80x80x32

print('h_pool1', h_pool1.shape)

# Weights and biases between hidden layer 1 and 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = weight_variable([64])

# Hidden layer 2
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, stride_size=4) + b_conv2)
# Max pooled hidden layer 2
h_pool2 = max_pool_2x2(h_conv2) # dim is 20x20x64

print('h_pool2', h_pool2.shape)

# Weights and biases between hidden layer 2 and 3
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

# Hidden layer 3
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
# Max pooled hidden layer 3
h_pool3 = max_pool_2x2(h_conv3) # dim is 5x5x128

print('h_pool3', h_pool3.shape)

h_pool1 (?, 80, 80, 32)
h_pool2 (?, 10, 10, 64)
h_pool3 (?, 3, 3, 128)


In [215]:
# Dense layer 1
W_d1 = weight_variable([3*3*128, 512])
b_d1 = weight_variable([512])

h_pool3_flat = tf.reshape(h_pool3, [-1, 3*3*128])
h_d1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_d1) + b_d1)

keep_prob = tf.placeholder(tf.float32)
h_d1_drop = tf.nn.dropout(h_d1, keep_prob)

# Dense layer 2
W_d2 = weight_variable([512, 1])
b_d2 = bias_variable([1])

# Output
y_pred = tf.matmul(h_d1_drop, W_d2) + b_d2

In [216]:
im = Image.open('images/0_61.png', 'r').convert('L')
pix = im.load()
im_array = []
for i in range(im.size[0]):
    for j in range(im.size[1]):
        im_array.append(pix[i,j])
im_array = np.expand_dims(np.array(im_array), axis=0) / 255

In [274]:
mse = tf.losses.mean_squared_error(y, y_pred)
train_step = tf.train.GradientDescentOptimizer(1e-7).minimize(mse)
init = tf.global_variables_initializer()

In [275]:
with tf.Session() as sess:
    sess.run(init)
    for n in range(1000):
        sess.run(train_step, feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 0.5
        })
        print(sess.run([mse], feed_dict={
            x: im_array,
            y: np.array([[105]]),
            keep_prob: 1
        }))

[10674.889]
[10626.413]
[10570.459]
[10519.36]
[10471.926]
[10436.684]
[10387.776]
[10342.439]
[10293.53]
[10240.965]
[10195.852]
[10135.164]
[10084.42]
[10022.99]
[9975.165]
[9920.333]
[9881.5615]
[9839.1318]
[9786.3887]
[9739.1816]
[9688.9814]
[9636.3896]
[9581.8818]
[9518.0693]
[9477.0605]
[9434.0566]
[9381.5127]
[9333.8887]
[9286.9287]
[9234.0283]
[9193.3252]
[9149.4551]
[9101.7256]
[9050.0459]
[9011.6631]
[8962.3047]
[8899.6748]
[8845.9668]
[8786.3838]
[8736.6777]
[8689.4248]
[8645.7793]
[8600.2842]
[8553.6572]
[8510.9805]
[8463.124]
[8413.4238]
[8369.1533]
[8321.459]
[8269.9287]
[8217.2197]
[8157.4146]
[8104.7979]
[8050.7891]
[7998.376]
[7950.0654]
[7889.5522]
[7829.4482]
[7778.3916]
[7721.2485]
[7671.3794]
[7626.6772]
[7564.417]
[7513.9727]
[7458.3398]
[7398.8179]
[7342.0771]
[7277.1982]
[7213.3306]
[7157.7329]
[7101.3423]
[7043.3652]
[6990.46]
[6925.9712]
[6865.6509]
[6809.9941]
[6757.3838]
[6696.6748]
[6630.3677]
[6566.0659]
[6505.0171]
[6446.9189]
[6389.4243]
[6326.2026]
[626

[0.078202911]
[0.032683626]
[0.058577206]
[0.14158221]
[0.25549248]
[1.0385885]
[0.35290155]
[0.0029392096]
[0.022324562]
[0.10019501]
[0.047614869]
[0.086596817]
[1.2701781]
[4.4822569]
[5.3805227]
[12.484118]
[12.9276]
[10.795711]
[10.746784]
[8.9936838]
[10.378974]
[8.9217558]
[8.0778913]
[10.543515]
[10.275363]
[8.3613596]
[4.8100224]
[7.4836926]
[12.46547]
[9.784708]
[11.744447]
[8.3546982]
[7.906466]
[7.1293383]
[5.6264644]
[5.0029039]
[3.501677]
[4.2089586]
[2.2544882]
[3.0068922]
[1.469679]
[2.914633]
[1.6873311]
[0.74640095]
[0.046901695]
[0.015463296]
[0.0015897139]
[0.24502772]
[0.24942812]
[0.33640212]
[0.66673094]
[0.29636639]
[0.15744801]
[0.2658534]
[0.093981691]
[0.040234108]
[0.16699125]
[0.80639499]
[0.92275584]
[0.16354235]
[0.095493972]
[0.22174822]
[0.85745239]
[0.24369264]
[1.1962557]
[0.85948825]
[0.56590402]
[0.62496966]
[1.1444259]
[1.3718072]
[1.9168153]
[4.8846354]
[7.397203]
[4.1934462]
[8.326539]
[10.708801]
[11.581499]
[10.692779]
[8.681159]
[6.9772124]
[5